In [12]:
pip install git+https://github.com/Wauplin/setfit@dont-use-deprecated-dataset-filter

  Cloning https://github.com/Wauplin/setfit (to revision dont-use-deprecated-dataset-filter) to c:\users\s7\appdata\local\temp\pip-req-build-lntk9s6x
  Resolved https://github.com/Wauplin/setfit to commit 74bfc124b79c3c2820c465817c5aaa34dafdc5a7
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for setfit: filename=setfit-1.1.0.dev0-py3-none-any.whl size=76021 sha256=7cbdfbdde440233b7caaf902662ae02bb722fd498fc645e718068dc9e10300d2
  Stored in directory: C:\Users\S7\AppData\Local\Temp\pip-ephem-wheel-cache-zifxn414\wheels\9b\c6\d4\c5980aa4b36b3f7032573a9b19b6881e9f375e15901cacbabf
Successfully built setfit
  Attempting uninstall: setfit
    Found existing installation: setfit 1.0.3
    Uninstalling setfit-1.0.3:
      Successfully uninstalled setfit-1.0.3
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/Wauplin/setfit 'C:\Users\S7\AppData\Local\Temp\pip-req-build-lntk9s6x'
  Running command git checkout -b dont-use-deprecated-dataset-filter --track origin/dont-use-deprecated-dataset-filter
  branch 'dont-use-deprecated-dataset-filter' set up to track 'origin/dont-use-deprecated-dataset-filter'.
  Switched to a new branch 'dont-use-deprecated-dataset-filter'


In [13]:
from setfit import SetFitModel, Trainer, SetFitTrainer, TrainingArguments, sample_dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import re
from datasets import Dataset
from sklearn.metrics import f1_score
from sentence_transformers.losses import CosineSimilarityLoss
import numpy as np 

In [14]:
df = pd.read_csv('10_reviews_No_stop.csv' , index_col=0)


In [15]:
df.head(2)

,BusinessId,review_id,rating,comment,creation_date
0,60202,364189,1,كنت اتعامل معهم اسعارهم ارخص لكن اخر مره حصلت ...,2024-05-11 05:56:29.327
1,60202,356203,1,عدم المصداقيه في اسعار المنتجات,2024-03-06 08:08:24.190


In [16]:
df.loc[:, 'label'] = np.where(df['rating'].isin([0, 1, 2]), 'Negative', 'Positive')

In [17]:
df.rename(columns={'comment' : 'text'} , inplace=True)

In [18]:
df = df[['text' , 'label']]

In [19]:
df

,text,label
0,كنت اتعامل معهم اسعارهم ارخص لكن اخر مره حصلت ...,Negative
1,عدم المصداقيه في اسعار المنتجات,Negative
2,المتجر سي جدا في مصداقيته في بيانات البضاعه وا...,Negative
3,لا يوجد,Negative
4,لا يوجد,Negative
...,...,...
62879,اكثر مايميز المتجر سرعه الرد والتوصيل السريع و...,Positive
62880,التعامل مع متجر توليب تجربه تستحق اعادتها اكثر...,Positive
62881,تجربتي معها جمييله وانسانه ذوق واخلاق ومتعاونه...,Positive
62882,متجر توليب جميل جدا والتعامل اجمل,Positive


In [20]:
df = df[df['label'].isin(['Positive', 'Negative'])]
df.head(2)

,text,label
0,كنت اتعامل معهم اسعارهم ارخص لكن اخر مره حصلت ...,Negative
1,عدم المصداقيه في اسعار المنتجات,Negative


In [21]:
df['text_length'] = df['text'].map(lambda a: len(a))
df = df[(df['text_length']<200) & (df['text_length']>10)]
df.shape

(36893, 3)

In [22]:
df['label'].value_counts()

label
Positive    35626
Negative     1267
Name: count, dtype: int64

In [23]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['text', 'label', 'text_length', '__index_level_0__'],
    num_rows: 36893
})

In [24]:
# Simulate the few-shot regime by sampling 8 examples per class
train_dataset = sample_dataset(dataset, label_column="label", num_samples=8)
eval_dataset = sample_dataset(dataset, label_column="label", num_samples=50)
test_dataset = sample_dataset(dataset, label_column="label", num_samples=50)

c:\Users\S7\anaconda3\envs\Twuaiq\Lib\site-packages\setfit\data.py:154: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.apply(lambda x: x.sample(min(num_samples, len(x)), random_state=seed))
c:\Users\S7\anaconda3\envs\Twuaiq\Lib\site-packages\setfit\data.py:154: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.apply(lambda x: x.sample(min(num_samples, len(x)), random_state=seed))
c:\Users\S7\anac

In [25]:
train_dataset

Dataset({
    features: ['text', 'label', 'text_length', '__index_level_0__'],
    num_rows: 16
})

In [26]:
eval_dataset

Dataset({
    features: ['text', 'label', 'text_length', '__index_level_0__'],
    num_rows: 100
})

In [27]:
test_dataset


Dataset({
    features: ['text', 'label', 'text_length', '__index_level_0__'],
    num_rows: 100
})

In [28]:
# Load a SetFit model from Hub
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
)

c:\Users\S7\anaconda3\envs\Twuaiq\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  url,
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [38]:
args = TrainingArguments(
    batch_size=16,
    num_epochs=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    metric="accuracy")

Map: 100%|██████████| 16/16 [00:00<00:00, 2285.49 examples/s]


In [39]:
# Train and evaluate
trainer.train()

***** Running training *****
  Num unique pairs = 144
  Batch size = 16
  Num epochs = 4
  Total optimization steps = 36
  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:01<?, ?it/s]

                                           

{'embedding_loss': 0.4134, 'learning_rate': 5e-06, 'epoch': 0.11}


 39%|███▉      | 124/319 [19:51<30:17,  9.32s/it]

In [9]:
import transformers
import huggingface_hub

print(transformers.__version__)
print(huggingface_hub.__version__)

4.44.2
0.24.6


In [8]:
!pip uninstall -y huggingface_hub
!pip install huggingface_hub==0.23.5

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2

  You can safely remove it manually.



  Using cached transformers-4.39.0-py3-none-any.whl.metadata (134 kB)
Using cached transformers-4.39.0-py3-none-any.whl (8.8 MB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.2 MB 2.8 MB/s eta 0:00:01
   ---------------------------- ----------- 1.6/2.2 MB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
